### Importing nessary libraries

In [3]:

#!pip install mlxtend
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
# çıktının tek bir satırda olmasını sağlar.
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

  Using cached mlxtend-0.20.0-py2.py3-none-any.whl (1.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2


In [4]:
df_ = pd.read_excel("C:/Users/turgu/OneDrive/Masaüstü/bootcamp/datasets/online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()

In [117]:
df = df_.copy()

In [118]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [119]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

def retail_data_prep(dataframe):
    """
    it prepares retail2 dataset for solution
    
    Parameters
    ----------
    dataframe: dataframe
    
    Returns
    -------
        returns dataframe ready for to work with solution
    
    """
    # drop na values
    dataframe.dropna(inplace=True)
    # if invoice column contains "c" that means order was cancelled
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    # "Quantity" column must be greater than 0
    dataframe = dataframe[dataframe["Quantity"] > 0]
    # "Price" column can't be less than 0
    dataframe = dataframe[dataframe["Price"] > 0]
    # get rid of from outliers 
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe

def check_df(dataframe, head=5, tail=5, detail=False):
    """
    It gives general sight of dataframe objects.
    Parameters
    ----------
    dataframe: dataframe
        dataframe from which variable(column) names are to be retrieved.
    head: int, default 5
        It determines that how many of the first rows will print.
    tail: int,  default 5
        It determines that how many of the last rows will print.
    detail: boolean, default False
        It gives quantiles values
    Returns
    -------
        this is function don't return anything.It just prints summarized values
    Examples
    ------
    import seaborn as sns
    df = sns.load_dataset("tips")
    print(check_df(df,detail=True))
    """

    print("##################### Index #####################")
    print(dataframe.index)
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(tail))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Total NA #####################")
    print(dataframe.isnull().sum().sum())
    if detail:
        print("##################### Quantiles #####################")
        print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
        print("##################### Describe #####################")
        print(dataframe.describe().T)

In [120]:
check_df(df, detail=True)

##################### Index #####################
RangeIndex(start=0, stop=541910, step=1)
##################### Shape #####################
(541910, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### Head #####################
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6 2010-12-01 08:26:00   2.55      17850.0  United Kingdom
1  536365     71053                  WHITE METAL LANTERN         6 2010-12-01 08:26:00   3.39      17850.0  United Kingdom
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8 2010-12-01 08:26:00   2.75      17850.0  United Kingdom
3 

In [121]:
df = retail_data_prep(df)

In [122]:
check_df(df, detail=True)

##################### Index #####################
Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,      8,      9,
            ...
            541900, 541901, 541902, 541903, 541904, 541905, 541906, 541907, 541908, 541909], dtype='int64', length=397885)
##################### Shape #####################
(397885, 8)
##################### Types #####################
Invoice                object
StockCode              object
Description            object
Quantity              float64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
##################### Head #####################
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER       6.0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom
1  536365     71053                  WHITE METAL LANTERN  

In [123]:
############################################
# Preparing (Invoice-Product Matrix)
############################################
# Turn our values to matematical form  
# We are interested with invoices


# Description   NINE DRAWER OFFICE TIDY   SET 2 TEA TOWELS I LOVE LONDON    SPACEBOY BABY GIFT SET
# Invoice
# 536370                              0                                 1                       0
# 536852                              1                                 0                       1
# 536974                              0                                 0                       0
# 537065                              1                                 0                       0
# 537463                              0                                 0                       1

In [124]:
df_fr = df[df['Country'] == "France"] # the dataset is too big so we are filtering the data

In [125]:
#df_fr = df[df['Description'] != "POSTAGE"] #

In [126]:
df_fr.groupby(['Invoice', 'Description']).agg({"Quantity": "sum"}).head(20) 

Quantity
Invoice Description                                  
536370   SET 2 TEA TOWELS I LOVE LONDON          24.0
        ALARM CLOCK BAKELIKE GREEN               12.0
        ALARM CLOCK BAKELIKE PINK                24.0
        ALARM CLOCK BAKELIKE RED                 24.0
        CHARLOTTE BAG DOLLY GIRL DESIGN          20.0
        CIRCUS PARADE LUNCH BOX                  24.0
        INFLATABLE POLITICAL GLOBE               48.0
        LUNCH BOX I LOVE LONDON                  24.0
        MINI JIGSAW CIRCUS PARADE                24.0
        MINI JIGSAW SPACEBOY                     24.0
        MINI PAINT SET VINTAGE                   36.0
        PANDA AND BUNNIES STICKER SHEET          12.0
        POSTAGE                                   3.0
        RED TOADSTOOL LED NIGHT LIGHT            24.0
        ROUND SNACK BOXES SET OF4 WOODLAND       24.0
        SET/2 RED RETROSPOT TEA TOWELS           18.0
        SPACEBOY LUNCH BOX                       24.0
        STARS GIFT TAPE                          24.0
        VINTAGE HEADS AND TAILS CARD GAME        24.0
        VINTAGE SEASIDE JIGSAW PUZZLES           12.0

In [127]:
df_fr.groupby(['Invoice', 'Description']).agg({"Quantity": "sum"}).unstack().iloc[0:5, 0:5] # pivot to table

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                 NaN                NaN                          NaN                      NaN                             24.0
536852                                 NaN                NaN                          NaN                      NaN                              NaN
536974                                 NaN                NaN                          NaN                      NaN                              NaN
537065                                 NaN                NaN                          NaN                      NaN                              NaN
537463                                 NaN                NaN                          NaN                      NaN                              NaN

In [128]:
# we have filled na values with 0
df_fr.groupby(['Invoice', 'Description']).agg({"Quantity": "sum"}).unstack().fillna(0).iloc[0:5, 0:5] 

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                 0.0                0.0                          0.0                      0.0                             24.0
536852                                 0.0                0.0                          0.0                      0.0                              0.0
536974                                 0.0                0.0                          0.0                      0.0                              0.0
537065                                 0.0                0.0                          0.0                      0.0                              0.0
537463                                 0.0                0.0                          0.0                      0.0                              0.0

In [129]:
# we are not insterested with the quantity so we have changed all numbers greater than zero to 1
df_fr.groupby(['Invoice', 'Description']).agg({"Quantity": "sum"}).unstack().fillna(0).applymap(
    lambda x: 1 if x > 0 else 0).iloc[0:5, 0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                   0                  0                            0                        0                                1
536852                                   0                  0                            0                        0                                0
536974                                   0                  0                            0                        0                                0
537065                                   0                  0                            0                        0                                0
537463                                   0                  0                            0                        0                                0

In [130]:
# I turned things a function we made it above
def create_invoice_product_df(dataframe, id=False):
    """we are not insterested with the quantity so we have changed all numbers greater than zero to 1"""
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)


In [131]:
fr_inv_pro_df = create_invoice_product_df(df_fr,id=True)
fr_inv_pro_df.head()

StockCode,10002,10120,10125,10135,11001,15036,15039,16012,16048,16218,16219,16225,16236,16237,16238,17174,20615,20617,20658,20665,20668,20674,20675,20676,20677,20679,20681,20682,20684,20685,20686,20702,20704,20711,20712,20713,20717,20718,20719,20723,20724,20725,20726,20727,20728,20749,20750,20751,20767,20777,20819,20832,20866,20914,20961,20963,20966,20967,20971,20972,20973,20974,20975,20977,20978,20979,20981,20983,20984,20992,20996,21012,21026,21027,21030,21034,21035,21039,21042,21054,21055,21056,21058,21059,21061,21062,21063,21064,21065,21068,21069,21070,21078,21080,21084,21086,21087,21088,21090,21094,21095,21096,21098,21108,21110,21111,21114,21116,21121,21122,21123,21124,21125,21126,21135,21136,21137,21154,21155,21156,21161,21162,21163,21164,21165,21166,21174,21175,21181,21188,21194,21195,21196,21197,21198,21199,21200,21201,21202,21204,21205,21206,21207,21208,21209,21210,21212,21213,21216,21217,21218,21219,21221,21224,21231,21232,21238,21239,21240,21242,21243,21244,21245,21246,21248,21249,21251,21257,21258,21259,21262,21284,21286,21294,21306,21326,21328,21329,21340,21351,21354,21355,21356,21358,21361,21365,21369,21371,21372,21374,21375,21377,21380,21381,21383,21385,21394,21395,21398,21399,21402,21403,21407,21408,21411,21424,21425,21426,21427,21428,21429,21430,21439,21442,21452,21463,21466,21467,21469,21470,21471,21472,21479,21481,21484,21485,21494,21495,21497,21498,21499,21500,21506,21507,21508,21509,21519,21523,21524,21527,21528,21531,21533,21535,21537,21539,21544,21555,21556,21557,21558,21559,21561,21563,21564,21576,21577,21578,21579,21581,21584,21591,21621,21622,21623,21624,21643,21648,21650,21658,21668,21669,21671,21672,21673,21675,21677,21678,21679,21680,21693,21696,21698,21700,21703,21704,21706,21707,21708,21709,21710,21711,21715,21716,21719,21721,21723,21724,21725,21731,21733,21739,21746,21747,21749,21754,21755,21756,21770,21774,21786,21787,21790,21791,21812,21813,21818,21819,21821,21828,21829,21832,21833,21843,21844,21845,21864,21865,21866,21871,21872,21875,21877,21880,21881,21882,21883,21884,21888,21889,21890,21891,21892,21894,21900,21901,21905,21906,21907,21908,21912,21913,21914,21915,21916,21917,21918,21922,21926,21928,21929,21930,21931,21932,21933,21934,21935,21936,21937,21943,21946,21947,21948,21949,21955,21967,21972,21974,21975,21976,21977,21980,21981,21982,21983,21985,21986,21987,21988,21989,21990,22023,22024,22025,22026,22027,22028,22029,22032,22035,22037,22041,22043,22044,22045,22046,22051,22052,22059,22063,22064,22065,22068,22070,22071,22072,22073,22076,22077,22078,22079,22080,22081,22082,22083,22084,22085,22086,22087,22088,22089,22090,22091,22093,22094,22099,22109,22110,22111,22112,22113,22114,22115,22116,22118,22120,22121,22124,22129,22130,22131,22132,22133,22134,22136,22138,22139,22141,22142,22144,22149,22150,22151,22153,22170,22174,22175,22176,22178,22179,22180,22181,22189,22191,22192,22193,22195,22197,22198,22199,22200,22203,22204,22207,22208,22209,22210,22211,22215,22219,22221,22222,22223,22224,22228,22231,22232,22236,22242,22243,22244,22254,22255,22262,22264,22269,22271,22272,22273,22274,22282,22285,22294,22295,22299,22300,22301,22302,22303,22305,22306,22309,22311,22312,22313,22314,22315,22318,22319,22320,22321,22322,22324,22325,22326,22327,22328,22329,22331,22332,22333,22334,22339,22340,22341,22343,22344,22346,22348,22349,22350,22352,22354,22355,22356,22357,22358,22361,22362,22365,22366,22367,22371,22372,22374,22375,22376,22377,22378,22379,22380,22381,22382,22383,22384,22385,22386,22389,22390,22393,22394,22396,22398,22399,22400,22402,22403,22406,22408,22411,22412,22413,22414,22416,22417,22418,22419,22422,22423,22425,22427,22431,22432,22433,22434,22435,22436,22437,22438,22439,22440,22443,22444,22445,22449,22451,22452,22453,22454,22456,22457,22459,22466,22467,22468,22469,22471,22472,22474,22475,22480,22482,22483,22485,22489,22491,22492,22493,22495,22498,22499,22500,22501,22502,22504,22505,22507,22508,22514,22515,22516,22517,22518,22519,22520,22521,22522,22523,22524,22525,22526,22529,22530,22531,22532,

In [132]:
def check_id(dataframe, stock_code):
    """
    take dataframe and stock code and return product name of stock_code
    """
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)

In [134]:
############################################
# Finding of Unity Rules
############################################

# Possibilities of all possible product associations
# a priori function is used, the min support value is determined(0.01) (only supports values are calculated)
frequent_itemsets = apriori(fr_inv_pro_df, min_support=0.01, use_colnames=True)

C:\Users\turgu\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [135]:
# sort by support value
frequent_itemsets.sort_values("support", ascending=False).head()

,support,itemsets
538,0.773779,(POST)
387,0.187661,(23084)
107,0.179949,(21731)
243,0.172237,(22554)
245,0.169666,(22556)


In [136]:
# min_support=0.01 AND min_threshold=0.01 same thing
# finding of unity rules
# calculation of confidence, lift ext.
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

In [137]:
# Postage is a freight value you can delete it
rules.sort_values("support", ascending=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8610,(POST),(23084),0.773779,0.187661,0.167095,0.215947,1.150730,0.021887,1.036077
8611,(23084),(POST),0.187661,0.773779,0.167095,0.890411,1.150730,0.021887,2.064267
3370,(POST),(21731),0.773779,0.179949,0.159383,0.205980,1.144661,0.020143,1.032784
3371,(21731),(POST),0.179949,0.773779,0.159383,0.885714,1.144661,0.020143,1.979434
7078,(POST),(22556),0.773779,0.169666,0.151671,0.196013,1.155290,0.020387,1.032771


In [138]:
rules.sort_values("lift", ascending=False).head(500)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1082358,"(22659, 23206, 22726, 22727)","(21558, 23254, 22326, 23199)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
254690,"(21558, 23254, 20750)","(22352, 23291)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1163665,"(21558, 23254, 22726, 20750)","(22727, 22352, 22326, 23291, 23199)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1163666,"(23291, 23254, 22726, 20750)","(22727, 22352, 21558, 22326, 23199)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
973889,"(23206, 22728, 20750, 22326, 23254)","(22659, 21558, 22727)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
...,...,...,...,...,...,...,...,...,...
1163544,"(22726, 20750, 22352, 22326, 23254)","(23199, 23291, 21558, 22727)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1163545,"(22726, 20750, 22352, 22326, 21558)","(23199, 23291, 23254, 22727)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1163546,"(22726, 20750, 22352, 22326, 23291)","(21558, 23199, 23254, 22727)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
973952,"(22326, 21558, 23206, 20750)","(22728, 22659, 23254, 22727)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf


In [139]:
############################################
# Recommending a product to customer 
############################################
# Example
# Customer , product_id: 22492

product_id = 22492
check_id(df, product_id)

['MINI PAINT SET VINTAGE ']


In [140]:
sorted_rules = rules.sort_values("lift", ascending=False)

In [141]:
sorted_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1082358,"(22659, 23206, 22726, 22727)","(21558, 23254, 22326, 23199)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
254690,"(21558, 23254, 20750)","(22352, 23291)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1163665,"(21558, 23254, 22726, 20750)","(22727, 22352, 22326, 23291, 23199)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
1163666,"(23291, 23254, 22726, 20750)","(22727, 22352, 21558, 22326, 23199)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf
973889,"(23206, 22728, 20750, 22326, 23254)","(22659, 21558, 22727)",0.010283,0.010283,0.010283,1.0,97.25,0.010177,inf


In [143]:

recommendation_list = []

for i, product in enumerate(sorted_rules["antecedents"]):
    for j in list(product):
       
        if j == product_id:
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])


recommendation_list[0:2]

[22326, 22556]

In [144]:
# rec_count  = How many product should i recommend
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

In [145]:
arl_recommender(rules, 22492, 3)

[22326, 22556, 22551]